In [1]:
#coding:utf-8

import re
import matplotlib.pyplot as plt

In [7]:
!wget https://nlp100.github.io/data/neko.txt
!mecab -o neko.txt.mecab neko.txt

--2021-07-12 01:38:46--  https://nlp100.github.io/data/neko.txt
nlp100.github.io (nlp100.github.io) をDNSに問いあわせています... 185.199.109.153, 185.199.110.153, 185.199.111.153, ...
nlp100.github.io (nlp100.github.io)|185.199.109.153|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 975789 (953K) [text/plain]
`neko.txt' に保存中

neko.txt            100%[===================>] 952.92K  --.-KB/s 時間 0.1s       

2021-07-12 01:38:46 (8.70 MB/s) - `neko.txt' へ保存完了 [975789/975789]



# Problem 30: Reading the result

mecabの出力は以下のようになっている<br>
表層形\t品詞,品詞細分類1,品詞細分類2,品詞細分類3,活用型,活用形,原形,読み,発音<br>
ex)ある	助動詞,*,*,*,五段・ラ行アル,基本形,ある,アル,アル

ファイルを読み込み、空白('')や改行('\n')以外の行を上から処理する。<br>
'EOS\n'が出てくるところまで各形態素を辞書型に格納し、リストに追加する。<br>
'EOS\n'が出てきたところで辞書型をリセットする。

In [7]:
morphs = []
sentences = []

with open('neko.txt.mecab', mode='r') as f:
    for line in f:
        if line != 'EOS\n':
            morph = line.split('\t')
            if morph[0]=='' or morph[0]=='\n':
                continue
            else:
                attr = morph[1].split(',')
                ans = {'surface':morph[0], 'base':attr[6], 'pos':attr[0], 'pos1':attr[1]}
                morphs.append(ans)
        else:
            sentences.append(morphs)
            morphs = []

In [12]:
#確認
sentences[:5]

[[{'surface': '一', 'base': '一', 'pos': '名詞', 'pos1': '数'}],
 [],
 [{'surface': '\u3000', 'base': '\u3000', 'pos': '記号', 'pos1': '空白'},
  {'surface': '吾輩', 'base': '吾輩', 'pos': '名詞', 'pos1': '代名詞'},
  {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'},
  {'surface': '猫', 'base': '猫', 'pos': '名詞', 'pos1': '一般'},
  {'surface': 'で', 'base': 'だ', 'pos': '助動詞', 'pos1': '*'},
  {'surface': 'ある', 'base': 'ある', 'pos': '助動詞', 'pos1': '*'},
  {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}],
 [{'surface': '名前', 'base': '名前', 'pos': '名詞', 'pos1': '一般'},
  {'surface': 'は', 'base': 'は', 'pos': '助詞', 'pos1': '係助詞'},
  {'surface': 'まだ', 'base': 'まだ', 'pos': '副詞', 'pos1': '助詞類接続'},
  {'surface': '無い', 'base': '無い', 'pos': '形容詞', 'pos1': '自立'},
  {'surface': '。', 'base': '。', 'pos': '記号', 'pos1': '句点'}],
 []]

# Problem 31: Verbs

In [21]:
verb_surfaces = set()

for sentence in sentences:
    for morph in sentence:
        if morph['pos']=='動詞':
            verb_surfaces.add(morph['surface'])

In [22]:
#確認
list(verb_surfaces)[:5]

['苦しむ', '和する', '収め', '這入る', '這い出そ']

In [25]:
len(verb_surfaces)

3893

# Problem 32: Base forms of verbs

In [23]:
verb_bases = set()

for sentence in sentences:
    for morph in sentence:
        if morph['pos']=='動詞':
            verb_bases.add(morph['base'])

In [24]:
#確認
list(verb_bases)[:5]

['苦しむ', '和する', 'ぐれる', '這入る', '浮かぶ']

In [26]:
len(verb_bases)

2300

# Problem 33: A of B

2つの名詞が「の」で連結されている<br>
-><br>
条件1: 連体化の助詞「の」<br>
条件2: 「の」の前後の品詞がどちらも名詞

各文を読み込んでいき、条件を満たすものがあればsetに追加する

In [44]:
AofB = set()

for sentence in sentences:
    for idx, morph in enumerate(sentence):
        if morph['surface']=='の' and morph['pos']=='助詞' and morph['pos1']=='連体化':
            if len(sentence)-idx!=1:
                A = sentence[idx-1]['pos']
                B = sentence[idx+1]['pos']
                if A=='名詞' and B=='名詞':
                    phase = sentence[idx-1]['surface']+'の'+sentence[idx+1]['surface']
                    AofB.add(phase)
            else:
                continue

In [46]:
#確認
list(AofB)[:5]

['僕の舌', '柳の下', 'ベオウルフの中', '小説の中', '泥棒の無謀']

# Problem 34: A B

In [288]:
AB_set = set()

for sentence in sentences:
    cnt = 0
    noun = ''
    for morph in sentence:
        if morph['pos']=='名詞':
            cnt += 1
            noun = ''.join([noun, morph['surface']])
        elif cnt > 1:
            AB_set.add(noun)
            cnt = 0
            noun = ''
        else:
            cnt = 0
            noun = ''
            
    if cnt > 1:
        AB_set.add(noun)

In [289]:
#確認
list(AB_set)[:5]

['愚考致候', '金田夫婦', '仁参', '水癲癇', '清和源氏']

# Problem 35: Frequency of words

# Problem 36: Top-ten frequent words

# Problem 37: Top-ten words co-occuring with 'neko'

# Problem 38: Histogram

# Problem 39: Zipf's law